# AI Camera
メインコンピュータから命令を受けるとJetson Nanoが起動して接続されたカメラから画像を取得してベッドに人がいるかどうかをAIによって判断し、その結果をファイルに書き込みメインコンピュータに読み取ってもらう。 <br>
ユーザーが起床したことが確認されるとAIによる処理を終了させて再起動をし、スタートアッププログラムによって再びメインコンピュータからの命令を受け取れるように待機する。

# Camera
## Logicool C270n
[Logicool C270n](https://www.logicool.co.jp/ja-jp/product/hd-webcam-c270n)はLogicoolが販売しているUSB接続のカメラである。このカメラはHD 720pで内蔵カメラのないデスクトップPCやテレビ通話などに利用されている。<br>
本プロジェクトでは使用するマイコンボードにはカメラ入力としてUSBとraspberry pi用の2つが用意されていたが汎用性の高いUSB接続にするため購入当時安価で納期の早かった本製品を採用した。 <br> 

# 画像差し込み希望 ( logicoolcamera.jpg )

# Jetson Nano<br>
[Jetson Nano](https://static6.arrow.com/aropdfconversion/1280937d98cee0191d01d75dc0cb547310d0fa57/jetson-nano-module-datasheet-us-1031771-r3-hr.pdf)とはNvidiaが開発したマイコンボードである。 <br>
本プロジェクトでは著者がAI開発においてLinuxを使用しているため、OSがLinuxでAIの並列処理に優れたGPUを搭載した本機を採用した。

# 画像差し込み希望 ( jetson nano.jpg )

## 必要なもの （購入品） <br>
* Jetson Nano <br>
* SDカード（32GB以上） <br>
* モニター(HDMIまたはDisplay port) <br>
* モニターケーブル <br>
* キーボード <br>
* マウス <br>
* WiFiドングル <br>

## セットアップ <br>
OS（ Ubuntu 18.04 LTS )&nbsp;&nbsp;:&nbsp;&nbsp;[JetPack 4.4.1](https://developer.nvidia.com/embedded/jetpack#install) <br>

1.　上記サイトのSDカードイメージファイルをダウンロードする <br>
2.　SDカードにダウンロードしたイメージファイルを「balenaEtcher」を用いて書き込む <br>
3.　SDカードをJetson Nanoに入れて電源アダプターを差し込んで起動する　（5V2A：microusb , 5V4A:DCプラグ）<br>
4.　ユーザー登録を行い、ネットワーク接続設定をしてインターネットと接続する <br>
5.　アップデートを行う <br>
<br>
    `sudo apt-get update` <br>
    `sudo apt-get upgrade` <br>

## Python 開発環境
### 必要なライブラリ
* OpenCV <br>
* numpy <br>
* time <br>
* argparse <br>
* math <br>
* datetime <br>
* json <br>
* subprocess <br>
* flask <br>
* glab <br>

# AI アルゴリズム　__*YOLO*__ <br>
[YOLO（You Only Look Once）](https://arxiv.org/pdf/2004.10934.pdf)とは物体検出手法の1つで画像を読み込んでCNNに通すことでAIが学習した物体を検出することができるアルゴリズムである。<br>
このアルゴリズムはLinux18.04やWindows10だけでなくRaspberry piやJetson Nanoのようなマイコンでも動作することが確認されたため本プロジェクトにて採用した。 <br>

## YOLO セットアップ <br>
1.　YOLOのオープンソースである [darknet](https://github.com/AlexeyAB/darknet) をダウンロードする <br>
 <br>
`git clone https://github.com/AlexeyAB/darknet` <br>
 <br>
2.　darknetのフォルダ内にあるMakefileを編集する <br>
 <br>
`cd darknet` <br>
`sudo vi Makefile` <br>
 <br>
* ___変更箇所___ (2020/11/25時点) <br>
1行目 ： GPU=1 <br>
2行目 ： CUDNN=1 <br>
3行目 ： CUDNN_HALF=1 <br>
4行目 ： OPENCV=1 <br>
7行目 ： LIBSO=1 <br> 
24行目 ： # -gencode arch=compute_61,code=[sm_61,compute_61] <br>
47行目 ： ARCH= -gencode arch=compute_53,code=[sm_53,compute_53] <br>
72行目 ： NVCC=/usr/local/cuda/bin/nvcc <br>
 <br>
3.　コンパイルする <br>
<br>
`make` <br>
<br>
エラーがなければコンパイル成功！

## YOLO デフォルト実行方法 <br>
セットアップが終わるとYOLOを利用することができるようになる。 <br>
実際に物体検出ができるかどうか確認にするため次のコマンドを実行する　<br>
<br>
`./darknet detect yolov4-tiny.cfg yolov4-tiny.weights data/dog.jpg` <br>
<br>
もしエラーがない場合、犬と自転車と車を検出結果が画面に表示されるようになっている。 <br>

## データセット＆CNN <br>
本プロジェクトではリアルタイムで人がベッドにいるかどうかを判断する必要がある。 <br>
そこでJetson Nanoにて13fpsを出せることが確認されている __yolov4-tiny.cfg__ を採用した。<br>
このファイルは"darknet/cfg/"にすでに存在するものである。<br>
また開発者がすでに公開された「人」を含む80種類の画像データセットで学習して得られた重みである __yolov4-tiny.weights__ にて人の検出を行った。<br>
このファイルは以下のコマンドを実行することで入手加納である。<br>
<br>
`wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights`

# カメラ接続テスト<br>
Jetson NanoのUSBポートにカメラのUSBを接続する<br>

## logicamera.py<br>
Jetson Nanoにてカメラがきちんと認識されて映像がくるかどうか確認するためのプログラム<br>
* 実行方法<br>
`python3 logicamera.py` <br>

このプログラムを実行するとモニターにカメラからの映像が表示される。 <br>
表示された画像を保存したい場合は表示された映像をアクティブにしてキーボードの「s」を入力すると"darknet/jphack/camera"に撮影した日時の名前の画像ファイルとして保存される。<br>
停止させたい場合は表示された映像をアクティブにしてキーボードの「q」を入力するまたは「Ctrl + c」で正常終了する。<br>
もし映像が出力されず以下のようなエラーが出た際はカメラのUSBを一度抜いて再びさすまたは別のUSBポートにさすことを推奨する。<br>
 <br>
`error: (-215) size.width>0 && size.height>0 in function cv::imshow` <br>

# YOLO　プログラム編集

AIカメラの役割である人の検出を行うために開発者が作成したプログラムを変更した。 <br>

## darknet_jphack.py
darknet.pyを編集したものである。 <br>
開発者が作成したものは学習データセットに含まれるものが映像から検出された時、その物体の予測値と位置を示す枠をモニターに表示する仕様となっていた。 <br>
しかし、80種類の物体を検出して表示すると非常にわかりにくいため「人」である __"person"__ が検出されたときだけ予測値と位置を表示するように変更した。 <br>

## jphack_test.py <br>
darknet_images.pyを編集したもので、任意のタイミングでカメラからの映像から人を検出するプログラム <br>
開発者が作成したものは1枚または複数枚の画像のパスを入力することでそれらの画像から物体検出を行う仕様となっていた。 <br>
そこで、カメラからの映像を取得して任意のタイミングで取得した映像に人が検出できるプログラムを作成した。 <br>
* 実行方法 <br>
`python3 jphack_camera_test.py` <br>

 エラーなく実行されるとカメラからの映像がモニターに表示される。 <br>
 YOLOによる物体検出を行いたい場合はTerminal外でキーボードの「s」を入力すると検出が行われて予測値がTerminalに表示される。 <br>
 また検出結果は"darknet/jphack/result_img"に撮影した日時の名前の画像ファイルとして保存されるため後ほど確認することが可能である。 <br>
 停止させたい場合はTerminal外でキーボードの「q」を入力するまたは「Ctrl + c」で正常終了する。

# AI Camera 動作確認テスト

## jphack_demo.py <br>
指定した時間の間、カメラからの映像に対して物体検出をしてリアルタイムでモニターに表示するプログラム <br>
jphack_camera_test.pyではユーザーが指定した瞬間のカメラ映像に対して物体検出をする仕様となっていたが、これではベッドから人が起き上がるまでを管理することができない。　<br>
そこで、カメラからの映像に対して常に物体検出ができるように変更した。<br>
* 実行方法 <br>
`python3 jphack_camera_test.py` <br>

エラーなく実行されるとカメラからの映像に対して物体検出をした結果がモニターにリアルタイムで表示される。 <br>
カメラからの映像を1フレームずつ保存したものが"darknet/jphack/video"にプログラム実行開始日時のファルダが生成されており、その中に撮影されたすべてのフレームがそれぞれ取得した日時の名前の画像ファイルとして保存される。 <br>
カメラからの映像を動画にしたものは"darknet/jphack/video"に生成されたプログラム実行開始日時のファルダの中に保存されている。 <br>
カメラからの映像から物体検出をした結果を1フレームずつ保存したものが"darknet/jphack/result"に撮プログラム実行開始日時のファルダが生成されており、その中にすべての検出結果がそれぞれ取得した日時の名前の画像ファイルとして保存される。 <br>
画面に表示された検出結果動画は"darknet/jphack/result"に生成されたプログラム実行開始日時のファルダの中に保存されている。 <br>

# 11月24日に撮影した結果とCUIの挙動を表示した動画を差し込む

# メインコンピュータとの連携
メインコンピュータの命令でAI Cameraが起動できるようにする

## Jetson Nanoの役割
メインコンピュータの命令を受けて以下のことを行う。 <br>
* AI Cameraの起動・物体検出開始 <br>
* 検出結果を渡す <br>
* AI Cameraの停止・再起動 <br>

## server.py
メインコンピュータの命令を受けて上記3つの処理を行うプログラム <br>
メインコンピュータとはローカルサーバーで繋がっており、サーバーとして命令を受け取る。 <br>
サーバー経由の命令でpythonファイルを実行させるため、pythonのウェブアプリケーションフレームワークであるflaskを採用した。 <br>
### start_yolo()
AI Cameraを起動させて物体検出を開始させる関数。 <br>
### return_result()
メインコンピュータからユーザーがベッドにいるかどうかを確認するように命令された際に結果を伝える関数 <br>
### overwrite_state()
メインコンピュータからユーザーが起床したため検出を終了するように命令された際にAI Cameraを停止させて再起動する関数

## jphack.py
サーバーによるAI Cameraの起動・停止や検出結果通達を行うプログラム  <br>
サーバーを用いてAI Cameraの起動・停止や検出結果通達を行えるようにするために__jphack_demo.py__を編集したものである。  <br>
これまでの実行方法とは異なり、メインコンピュータからのサーバーからの命令であるserver.pyの__start_yolo()__によってAI Cameraの起動が求められると起動準備をして"state.json"が書き換えられることで検出を開始する仕様となっている。 <br>
また、1秒間隔で人がいるかどうかを判断した結果を"jphack.json"に書き込むことで、server.pyの__return_result()__によってメインコンピュータが検出結果を求めらた際に結果を送信できるようにしている。 <br>
そして、server.pyの__overwrite_state()__によって"state.json"が書き換えられることで検出を終了して再起動する。

## サーバー経由での実行方法
メインコンピュータからの命令で実行するためには以下のコマンドを実行する必要がある。 <br>
darknetのカレントディレクトリにて <br>
 <br>
`FLASK_APP=server.py flask run --host=0.0.0.0` <br>

## 再起動させる理由
上述のように本機はメインコンピュータからの終了命令を受け取ると再起動する仕様となっている。 <br>
これは再起動せずに実行した場合、メモリ不足によって物体検出の速度が著しく低下してリアルタイム処理ができないためである。 <br>
そこで、本機を再起動させて上記のコマンドを実行することでメモリ不足問題を回避した。

## 再起動後の対応方法
初めての起動の際はコマンドをキーボード入力することができるが、これを自動で行えるようにしなければユーザーが毎回手動入力することになり、手間がかかる。 <br>
そこで、起動した際ににサーバー経由実行に必要なコマンドを入力するプログラムを入れることで自動入力できるようにスタートアップサービスを変更した。

# スタートアップサービス
Ubuntu 18.04 LTSではいくつかのスタートアップサービスの変更方法が紹介されているが、本プロジェクトでは__etc/rc.local__によるスタートアップを採用した。　<br>

## startup.sh
起動時に実行したいコマンドをこのファイル内に記述している。 <br>
本プロジェクトでは起動時にdarknetのカレントディレクトリへ移動してサーバー経由実行となっているため、shellファイル内にカレントディレクトリへの移動とサーバー経由実行コマンドの2つを改行して書き込んでいる。 <br>
なお、このファイルはホームディレクトリに作成する。 <br>
* ファイル作成・編集方法 <br>
`sudo vi startup.sh` <br>

作成したshellファイルは読み込みしか対応できていないため実行できるようにするため以下のコマンドを実行する。<br>
* shellファイル有効化 <br>
`sudo chmod 755 startup.sh` <br>

## etc/rc.local
スタートアップサービスにて、上記で作成したshellファイルを実行するためにshellファイルのパスを保存するためのlocalファイルを作成する。 <br>
* 作成するディレクトリ： __/etc__<br>
* ファイル作成・編集方法<br>
`sudo vi etc/rc.local` <br>

このファイルには実行したいshellファイルのpathを記述する。 <br>
ファイル作成時、1行目に__#!/bin/sh__を入れないとエラーが出ることがあるため注意が必要である。　<br>
このファイルも有効化するために次のコマンドを実行する必要がある。 <br>
* localファイル有効化 <br>
`sudo chmod 755 startup.sh` <br>

## rc-local.service
上記で作成したshellファイルをスタートアップサービスとして実行するために必要なserviceファイルを作成する。 <br>
* 作成するディレクトリ： __/etc/systemd/system__<br>
* ファイル作成・編集方法<br>
`sudo vi etc/etc/systemd/system/rc-local.service` <br>

このファイルは大きく3つで構成されている。 <br>
* Unit <br>
* Service <br>
* Install <br>

それぞれの要素においてたくさんの引数があり、引数を使うことで定期的にプログラムを実行することやある条件を満たすまで何もできないようにすることができる。 <br>
本プロジェクトではJetson Nanoが再起動した際に1度だけshellファイルに記述したものを実行することでユーザーが本機に触れることなく永久的に利用することができるため、引数は最低限しか使わないようにした。 <br>
なお、ファイル作成時、1行目に__# vi /etc/systemd/system/rc-local.service__を入れないとエラーが出ることがあるため注意が必要である。
このファイルも有効化するために次のコマンドを実行する必要がある。
* serviceファイル有効化 <br>
`sudo chmod +x /etc/systemd/system/rc-local.service` <br>

## スタートアップサービスの設定
上記で作成した3つのファイルを用いて起動時にshellファイル内に記述したものを実行できるように設定する。 <br>
* スタートアップサービスの有効化　<br>
`sudo systemctl enable rc-local` <br>
<br>
* スタートアップサービスの開始　<br>
`sudo systemctl start rc-local`

ここまででエラーが出なければスタートアップサービスとして登録されたことになる。<br>
<br>
現在の状態は次のコマンドで確認できる。 <br>
* スタートアップサービスの状況　<br>
`sudo systemctl status rc-local`<br>

また次のコマンドでスタートアップサービスを終了させることができる。<br>
* スタートアップサービスの終了<br>
`sudo systemctl stop rc-local`<br>
<br>
* スタートアップサービスの無効化<br>
`sudo systemctl disable rc-local`<br>

# 開発ポイント
1. YOLO
2. サーバー化
3. スタートアップサービス

## 1. YOLO
大学の研究にてAIによる画像分類や異常検知を担当していたが、今年からYOLOによる物体検出を開始し、自前の画像データに対して正解ラベルを作成して学習を行った経験があった。 <br>
しかし、出力結果をもとに実行する内容を変えたり、任意のタイミングで終了するなどのプログラム作成には取り組んではいなかった。 <br>
また、研究ではGPUを搭載したデスクトップパソコンを使用して開発を行っているが、本プロジェクトの目的である小型化には向いていなかったため、以前より注目していたJetson Nanoを今回のハッカソンのために購入して使えるようにした。 <br>

新しいタスクと新しいデバイスでYOLOを利用するためにはYOLOの創設者が公開しているものでは対応できないと判断したため、本プロジェクト内にてJetson Nanoに対応したYOLOのオープンソースを探し、その開発者が公開していたpythonファイルに注目したところ、プログラムを編集することで利用できることが予想されたためYOLOを採用した。 <br>

実際に環境構築を行い、通常のコマンドが実行できることを確認してからpythonプログラムの編集に取り掛かったが、YOLOをC++で開発したものをpythonで実行できる形に変更していたため1つ1つの関数の処理が何をしているのか理解するのに時間を要した。 <br>
また、画像1枚の処理をすることは開発者のプログラムと大きな差はないが、カメラからの映像に対してリアルタイムで処理して表示することに関しては処理の速さが求められたため、エラーによるプログラム終了を回避しながら処理数をできるだけ少なくしてメモリに余裕をもたせることでリアルタイム処理を可能にした。 <br>

## 2. サーバー化
本機開発においてJetson Nanoのサーバー化に非常に時間を要した。 <br>
そもそもサーバー化することがなにかをよくわからなかったため、サーバーについて調べてできることを少し学んだ。 <br>
開発では、メインコンピュータからの命令を受けて実行するために作成したプログラムをどうすればよいか考え、関数化とjsonファイルの値による操作分岐によって解決することができた。 <br>
また、AI Cameraを起動してメインコンピュータから終了命令が来るまで検出を繰り返した後に本機を停止させるが、2日目再び起動させるとメモリ不足を引き起こしてリアルタイム処理ができない状態になったが、再起動することでメモリ不足を回避して正常に動作できるようにした。

## 3. スタートアップサービス
上述のサーバー化の中で再起動させることでメモリ不足を回避することができたが、再起動したことによってメインコンピュータからの命令を受け取れない状態になってしまった。 <br>
そこで、再起動時にサーバー化するように改良することで何回でも本機を起動できると考えた。 <br> 
研究にてLinuxを使用していたが、スタートアップサービスを利用する必要はなかったため、どうすればいいかわからなかった。 <br>
本機開発において2つの方法を使用した。 <br>
1. crontab
2. rc.local & rc_local service

### 1. crontab
インターネットでスタートアップ時に指定したプログラムを実行する方法を探していたところ、__crontab__による方法が数多く見られた。 <br>
そこで、大学の研究で使用しているLinuxのパソコンにて設定してみたところ、起動時に指定したプログラムが実行されたため問題なく使えることが確認された。 <br>
しかし、Jetson Nanoで同じように設定して再起動しても何も変わることはなかった。エラー表示などもなかったため対処方法がわからず振り出しに戻った。

### 2. rc.local & rc_local service 
上述の方法ではうまくいかなかったため、__etc/rc.local__による方法を試した。 <br>
こちらの方法は時間の都合上、大学のパソコンではテストすることなくいきなりJetson Nanoにて設定しようとしたが、エラーが頻発した。　<br>
「ファイルの実行権限がない」・「ライブラリのバージョンが異なる」・「起動時に0.2秒ほどだけ見えるserviveファイルに対する引数に対するエラー」など様々なエラーに対応した。 <br>
これらのエラーに対応したことにより、スタートアップサービスの実現に成功した。 <br>
なお、Jetson Nanoは初期セットアップとアップデート、pythonやライブラリの利用に必要なものだけをダウンロードしただけのものなので他の方法ですでに利用されている方はこれらのエラーに遭遇しないことが予想される。

# コメント
以上が本機開発者からのデバイスのセットアップおよびプログラムの説明になります。 <br>
Jetson NanoはGPUを搭載した小型マイコンということで研究機関や商品開発などで幅広く利用されているようです。 <br>
Jetson Nanoを利用した著者としましても非常に使いやすく汎用性の高いものであると実感し、今後は四輪車やドローンの開発などに活用していくことを検討しています。 <br>
こちらをご覧になった皆様がJetson NanoやAIに興味を持っていただき、各々の研究開発や趣味の多様化などに貢献できますと幸いです。 <br>
<br>
最後までお目通し頂き、誠にありがとうございました。